In [ ]:
# ==============================================================================
# 1. INSTALASI LIBRARY YANG DIPERLUKAN DAN IMPORT
# ==============================================================================
# Instal library Sastrawi untuk stemming Bahasa Indonesia
# dan openpyxl untuk membaca file Excel (.xlsx)
!pip install Sastrawi openpyxl

import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

print("Library yang dibutuhkan telah siap.")

Library yang dibutuhkan telah siap.


In [ ]:
# ==============================================================================
# 2. MUAT DATASET DARI FILE .XLSX
# ==============================================================================
# Path file yang Anda berikan
file_path = '/content/sample_data/daftarcampaign.xlsx'

try:
    # Menggunakan pd.read_excel() untuk membaca file Excel.
    # Tidak perlu parameter encoding untuk file .xlsx
    df = pd.read_excel(file_path)
    print("Dataset .xlsx berhasil dimuat.")
    print("Contoh 5 baris pertama dari data Anda:")
    print(df.head())
except FileNotFoundError:
    print(f"Error: File '{file_path}' tidak ditemukan.")
    print("Pastikan file sudah ada di lokasi tersebut. Anda bisa memeriksanya di panel file sebelah kiri.")
except Exception as e:
    print(f"Terjadi error saat membaca file Excel: {e}")

# (Lanjutkan dengan kode preprocessing dan seterusnya dari skrip sebelumnya...)

Dataset .xlsx berhasil dimuat.
Contoh 5 baris pertama dari data Anda:
   No                                              Judul  \
0   1      [DARURAT] Bangun Lagi Rumah Sakit untuk GAZA!   
1   2  🚨URGENT 🚨 Gaza Tanpa Masjid, Dimana Mereka Ber...   
2   3  🚨 DARURAT!! Gaza Kembali Diserang, Mereka Butu...   
3   4  Zakat untuk Anak Yatim & Dhuafa: Wujudkan Keba...   
4   5              Sedekah Jariyah : Muliakan Guru Ngaji   

                                           Deskripsi        Yayasan  \
0  Serangan brutal Zionis Israel porak porandakan...  Dompet Dhuafa   
1  Gaza tanpa masjid, di mana mereka bersujud? 💔 ...  Dompet Dhuafa   
2  Hanya dalam 50 jam, sebanyak 710 warga Palesti...  Dompet Dhuafa   
3  Saatnya Berbagi! Tunaikan zakat untuk yatim & ...  Dompet Dhuafa   
4  Raih pahala jariyah dengan muliakan guru ngaji...  Dompet Dhuafa   

      Kategori  
0  Kemanusiaan  
1  Kemanusiaan  
2  Kemanusiaan  
3        Zakat  
4       Sosial  


In [ ]:
# ==============================================================================
# 3. PRA-PEMROSESAN DATA TEKS (PREPROCESSING)
# ==============================================================================
# Menggabungkan fitur teks (Judul dan Deskripsi) menjadi satu kolom 'content'
# untuk dianalisis. Ini akan menangkap makna dari kedua kolom tersebut.
# Pastikan nama kolom 'Judul' dan 'Deskripsi' sesuai dengan file Excel Anda.
print("\nMemulai tahap pra-pemrosesan data...")
df['content'] = df['Judul'] + ' ' + df['Deskripsi']
print("Kolom 'Judul' dan 'Deskripsi' telah digabung menjadi kolom 'content'.")

# Inisialisasi stemmer dan stopword remover untuk Bahasa Indonesia
factory_stemmer = StemmerFactory()
stemmer = factory_stemmer.create_stemmer()

factory_stopword = StopWordRemoverFactory()
stopword_remover = factory_stopword.create_stop_word_remover()

def preprocess_text(text):
    """
    Fungsi untuk membersihkan, menghapus stopword, dan melakukan stemming pada teks.
    """
    if not isinstance(text, str):
        return ""
    # 1. Case Folding: Mengubah teks menjadi huruf kecil
    text = text.lower()
    # 2. Menghapus karakter non-alfanumerik (kecuali spasi)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # 3. Stopword Removal
    text = stopword_remover.remove(text)
    # 4. Stemming
    text = stemmer.stem(text)
    return text

# Terapkan fungsi preprocessing ke kolom 'content'
print("Membersihkan teks (case folding, stopword removal, stemming)...")
df['content_cleaned'] = df['content'].apply(preprocess_text)
print("Proses pra-pemrosesan selesai.")
print("\nContoh hasil pra-pemrosesan:")
print(df[['Judul', 'content_cleaned']].head())


Memulai tahap pra-pemrosesan data...
Kolom 'Judul' dan 'Deskripsi' telah digabung menjadi kolom 'content'.
Membersihkan teks (case folding, stopword removal, stemming)...
Proses pra-pemrosesan selesai.

Contoh hasil pra-pemrosesan:
                                               Judul  \
0      [DARURAT] Bangun Lagi Rumah Sakit untuk GAZA!   
1  🚨URGENT 🚨 Gaza Tanpa Masjid, Dimana Mereka Ber...   
2  🚨 DARURAT!! Gaza Kembali Diserang, Mereka Butu...   
3  Zakat untuk Anak Yatim & Dhuafa: Wujudkan Keba...   
4              Sedekah Jariyah : Muliakan Guru Ngaji   

                                     content_cleaned  
0  darurat bangun rumah sakit gaza serang brutal ...  
1  urgent gaza masjid sujud gaza masjid mana suju...  
2  darurat gaza serang butuh hanya 50 jam banyak ...  
3  zakat anak yatim dhuafa wujud bahagia di ramad...  
4  sedekah jariyah mulia guru ngaji raih pahala j...  


In [ ]:
# ==============================================================================
# 4. PEMBUATAN MODEL (TF-IDF VECTORIZER)
# ==============================================================================
# Membuat objek TfidfVectorizer. Ini adalah "model" Content-Based Filtering kita.
# Model ini akan belajar kosakata dan bobot IDF dari seluruh data kampanye.
tfidf_vectorizer = TfidfVectorizer()

# Melakukan "training" model: mempelajari kosakata dan menghitung bobot IDF,
# lalu mentransformasi teks menjadi matriks TF-IDF.
print("\nMembuat model TF-IDF dan mentransformasi data...")
tfidf_matrix = tfidf_vectorizer.fit_transform(df['content_cleaned'])
print("Pembuatan matriks TF-IDF selesai.")
print(f"Bentuk matriks TF-IDF: {tfidf_matrix.shape} (Jumlah Kampanye, Jumlah Kata Unik)")


Membuat model TF-IDF dan mentransformasi data...
Pembuatan matriks TF-IDF selesai.
Bentuk matriks TF-IDF: (177, 898) (Jumlah Kampanye, Jumlah Kata Unik)


In [ ]:
# ==============================================================================
# 5. MENYIMPAN MODEL (.pkl)
# ==============================================================================
# Sesuai revisi, kita simpan objek vectorizer dan matriks TF-IDF
# agar bisa digunakan nanti di aplikasi web tanpa perlu training ulang.

# Menyimpan TF-IDF Vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)
print("\nModel TF-IDF Vectorizer telah disimpan sebagai 'tfidf_vectorizer.pkl'")

# Menyimpan Matriks TF-IDF
with open('tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)
print("Matriks TF-IDF telah disimpan sebagai 'tfidf_matrix.pkl'")
print("Anda bisa mengunduh kedua file .pkl ini dari panel file di sebelah kiri Colab.")


Model TF-IDF Vectorizer telah disimpan sebagai 'tfidf_vectorizer.pkl'
Matriks TF-IDF telah disimpan sebagai 'tfidf_matrix.pkl'
Anda bisa mengunduh kedua file .pkl ini dari panel file di sebelah kiri Colab.


In [ ]:
# ==============================================================================
# 6. MENGHITUNG KEMIRIPAN (COSINE SIMILARITY)
# ==============================================================================
# Menghitung matriks cosine similarity dari matriks TF-IDF.
# Hasilnya adalah matriks (n_kampanye x n_kampanye) dimana setiap sel (i, j)
# berisi skor kemiripan antara kampanye i dan kampanye j.
print("\nMenghitung matriks Cosine Similarity...")
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("Perhitungan Cosine Similarity selesai.")
print(f"Bentuk matriks Cosine Similarity: {cosine_sim.shape}")


Menghitung matriks Cosine Similarity...
Perhitungan Cosine Similarity selesai.
Bentuk matriks Cosine Similarity: (177, 177)


In [ ]:
# ==============================================================================
# 7. FUNGSI DAN DEMONSTRASI REKOMENDASI
# ==============================================================================
# Buat series untuk memetakan judul kampanye ke indeksnya agar mudah dicari
indices = pd.Series(df.index, index=df['Judul']).drop_duplicates()

def get_recommendations(title, cosine_sim_matrix, dataframe):
    """
    Fungsi untuk mendapatkan rekomendasi berdasarkan judul kampanye.
    """
    if title not in indices:
        return f"Error: Judul '{title}' tidak ditemukan dalam dataset."

    # Dapatkan indeks dari judul kampanye yang diberikan
    idx = indices[title]

    # Dapatkan skor kemiripan dari semua kampanye dengan kampanye tersebut
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # Urutkan kampanye berdasarkan skor kemiripan (dari tertinggi ke terendah)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Ambil skor dari 10 kampanye paling mirip (abaikan yang pertama karena itu kampanye itu sendiri)
    sim_scores = sim_scores[1:11]

    # Dapatkan indeks kampanye yang direkomendasikan
    campaign_indices = [i[0] for i in sim_scores]

    # Kembalikan DataFrame kecil berisi judul dan skor kemiripan
    recommendations = dataframe[['Judul', 'Kategori']].iloc[campaign_indices].copy()
    recommendations['Similarity Score'] = [round(score[1], 4) for score in sim_scores]
    return recommendations.reset_index(drop=True)

In [ ]:
# --- DEMONSTRASI SESUAI PERMINTAAN REVISI ---

print("\n" + "="*60)
print("         DEMONSTRASI HASIL REKOMENDASI        ")
print("="*60)

# Skenario 1: Donatur 1 (Suka kampanye Bencana)
print("\nSKENARIO 1: DONATUR 1")
try:
    # Cari kampanye pertama yang mengandung kategori 'Bencana'
    campaign_bencana = df[df['Kategori'].str.contains('Bencana', case=False, na=False)].iloc[0]['Judul']
    print(f"Donatur 1 sebelumnya berdonasi untuk kampanye bencana: '{campaign_bencana}'")
    print("------------------------------------------------------------------")
    print("Rekomendasi kampanye serupa untuk Donatur 1:")
    recommendations_donatur1 = get_recommendations(campaign_bencana, cosine_sim, df)
    print(recommendations_donatur1)
except IndexError:
    print("\nTidak ditemukan kampanye dengan kategori 'Bencana' dalam dataset untuk demonstrasi Donatur 1.")
except Exception as e:
    print(f"\nTerjadi error saat mencari rekomendasi untuk Donatur 1: {e}")


# Skenario 2: Donatur 2 (Suka kampanye Kemanusiaan dan Zakat)
print("\n\nSKENARIO 2: DONATUR 2")
try:
    # Rekomendasi berdasarkan riwayat donasi Kemanusiaan
    campaign_kemanusiaan = df[df['Kategori'].str.contains('Kemanusiaan', case=False, na=False)].iloc[0]['Judul']
    print(f"Donatur 2 sebelumnya berdonasi untuk kampanye kemanusiaan: '{campaign_kemanusiaan}'")
    print("------------------------------------------------------------------")
    print("Rekomendasi kampanye serupa (berdasarkan riwayat kemanusiaan):")
    recommendations_donatur2_kemanusiaan = get_recommendations(campaign_kemanusiaan, cosine_sim, df)
    print(recommendations_donatur2_kemanusiaan)
except IndexError:
    print("\nTidak ditemukan kampanye dengan kategori 'Kemanusiaan' dalam dataset untuk demonstrasi Donatur 2.")
except Exception as e:
    print(f"\nTerjadi error saat mencari rekomendasi Kemanusiaan untuk Donatur 2: {e}")

try:
    # Rekomendasi berdasarkan riwayat donasi Zakat
    campaign_zakat = df[df['Kategori'].str.contains('Zakat', case=False, na=False)].iloc[0]['Judul']
    print(f"\nDonatur 2 juga berdonasi untuk kampanye zakat: '{campaign_zakat}'")
    print("------------------------------------------------------------------")
    print("Rekomendasi kampanye serupa (berdasarkan riwayat zakat):")
    recommendations_donatur2_zakat = get_recommendations(campaign_zakat, cosine_sim, df)
    print(recommendations_donatur2_zakat)
except IndexError:
    print("\nTidak ditemukan kampanye dengan kategori 'Zakat' dalam dataset untuk demonstrasi Donatur 2.")
except Exception as e:
    print(f"\nTerjadi error saat mencari rekomendasi Zakat untuk Donatur 2: {e}")


         DEMONSTRASI HASIL REKOMENDASI        

SKENARIO 1: DONATUR 1
Donatur 1 sebelumnya berdonasi untuk kampanye bencana: 'Tolong! Semeru Erupsi, 1.979 Warga Mengungsi'
------------------------------------------------------------------
Rekomendasi kampanye serupa untuk Donatur 1:
                                               Judul     Kategori  \
0  DARURAT! BANTU WARGA TERDAMPAK ERUPSI GUNUNG L...  Kemanusiaan   
1  Darurat! Bantu Warga Terdampak Banjir Lahar Di...  Kemanusiaan   
2                     SOLIDARITAS BANJIR JAWA TENGAH  Kemanusiaan   
3        Kirim Bantuan Medis Untuk Pengungsi Cianjur  Kemanusiaan   
4       URGENT! BANTU WARGA TERDAMPAK BANJIR LAMPUNG  Kemanusiaan   
5  17 orang meninggal dunia, 50 lainnya luka bera...  Kemanusiaan   
6               Bantu Korban Terdampak Gempa Cianjur  Kemanusiaan   
7  [MENDESAK] Jabodetabek dikepung banjir hingga ...  Kemanusiaan   
8  Mau Didoakan Malaikat? Yuk Sedekah Subuh Janga...       Sosial   
9  SOLIDARITAS BANTU PENY

In [ ]:
import pandas as pd
import sklearn
import numpy

print("--- Versi Library di Google Colab ---")
print(f"Versi Pandas: {pd.__version__}")
print(f"Versi Scikit-learn: {sklearn.__version__}")
print(f"Versi Numpy: {numpy.__version__}")

--- Versi Library di Google Colab ---
Versi Pandas: 2.2.2
Versi Scikit-learn: 1.6.1
Versi Numpy: 2.0.2
